# Libraries

In [ ]:
# Installazione delle librerie necessarie con specifiche per l'uso della GPU
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install openpyxl
!pip install pandas

In [ ]:
# Importing necessary libraries
import spacy
import json
import os
import re
import csv
import openpyxl
import pandas as pd
import numpy as np
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from google.colab import drive
drive.mount('/content/drive')

# Corpus Analysis

In [ ]:
# Load the Excel file
data_path = '/content/drive/My Drive/Llama2_Translation_LNC/interaction-corpus.xlsx'
data = pd.read_excel(data_path)

# Display the first few rows of the dataset to understand its structure
print(data.head())

In [ ]:
# Filter the data for system responses only
system_responses = data[data['Participant'] == 'S']

# Count the types of errors for system responses
system_error_counts = system_responses['Errors_GOLD'].value_counts()
system_error_counts

In [ ]:
# Filter the data for system responses only
user_responses = data[data['Participant'] == 'U']

# Count the types of errors for system responses
system_error_counts = user_responses['Errors_GOLD'].value_counts()
system_error_counts

Escludiamo:
1. Off-topic
3. Non-understandable
4. Non-cooperativity
5. Repetition

## Topic Change

In [ ]:
# Identify rows where the system response has a "Topic change" error
topic_change_indices = system_responses[system_responses['Errors_GOLD'] == 'Topic change'].index

# Get the corresponding user inputs just before the system responses with "Topic Change"
user_inputs_before_topic_change = data.loc[topic_change_indices - 1]

# Define the accepted user errors
accepted_user_errors = ['Ignoring question/feedback', 'Grammatical error', 'Lack of information', 'Ill-formed']

# Filter the user inputs to include only those with the specified Errors_GOLD
filtered_user_inputs_tc = user_inputs_before_topic_change[
    user_inputs_before_topic_change['Errors_GOLD'].isin(accepted_user_errors) | user_inputs_before_topic_change['Errors_GOLD'].isnull()
]
# Extract only the text of the queries
final_user_queries_tc = filtered_user_inputs_tc['Text'].sample(15).tolist()
final_user_queries_tc

In [ ]:
# Calcolare la lunghezza dell'array matched_queries_updated
array_length = len(filtered_user_inputs_tc)
print("La lunghezza dell'array è:", array_length)

## Straight Wrong Response

In [ ]:
# Identify rows where the system response has a "Straight wrong response" error
straight_wrong_response_indices = system_responses[system_responses['Errors_GOLD'] == 'Straight wrong response'].index

# Get the corresponding user inputs just before the system responses with "Straight wrong response"
user_inputs_before_straight_wrong_response = data.loc[straight_wrong_response_indices - 1]

# Define the accepted user errors
accepted_user_errors = ['Ignoring question/feedback', 'Grammatical error', 'Lack of information', 'Ill-formed']

# Filter the user inputs to include only those with the specified Errors_GOLD
filtered_user_inputs_swr = user_inputs_before_straight_wrong_response[
    user_inputs_before_straight_wrong_response['Errors_GOLD'].isin(accepted_user_errors) | user_inputs_before_straight_wrong_response['Errors_GOLD'].isnull()
]
# Extract only the text of the queries
final_user_queries_swr = filtered_user_inputs_swr['Text'].sample(10).tolist()
final_user_queries_swr

In [ ]:
# Calcolare la lunghezza dell'array matched_queries_updated
array_length = len(filtered_user_inputs_swr)
print("La lunghezza dell'array è:", array_length)

In [ ]:
filtered_user_inputs_swr['Text'].tolist()

## Indirect Response

In [ ]:
# Identify rows where the system response has a "Indirect response" error
indirect_response_indices = system_responses[system_responses['Errors_GOLD'] == 'Indirect response'].index

# Get the corresponding user inputs just before the system responses with "Indirect response"
user_inputs_before_indirect_response = data.loc[indirect_response_indices - 1]

# Define the accepted user errors
accepted_user_errors = ['Ignoring question/feedback', 'Grammatical error', 'Lack of information', 'Ill-formed']

# Filter the user inputs to include only those with the specified Errors_GOLD
filtered_user_inputs_ir = user_inputs_before_indirect_response[
    user_inputs_before_indirect_response['Errors_GOLD'].isin(accepted_user_errors) | user_inputs_before_indirect_response['Errors_GOLD'].isnull()
]
# Extract only the text of the queries
final_user_queries_ir = filtered_user_inputs_ir['Text'].sample(8).tolist()
final_user_queries_ir

In [ ]:
# Calcolare la lunghezza dell'array matched_queries_updated
array_length = len(filtered_user_inputs_ir)
print("La lunghezza dell'array è:", array_length)

In [ ]:
filtered_user_inputs_ir['Text'].tolist()

## Excess of Information

In [ ]:
# Identify rows where the system response has a "Excess of information" error
excess_of_information_indices = system_responses[system_responses['Errors_GOLD'] == 'Excess of information'].index

# Get the corresponding user inputs just before the system responses with "Excess of information"
user_inputs_before_excess_of_information = data.loc[excess_of_information_indices - 1]

# Define the accepted user errors
accepted_user_errors = ['Ignoring question/feedback', 'Grammatical error', 'Lack of information', 'Ill-formed']

# Filter the user inputs to include only those with the specified Errors_GOLD
filtered_user_inputs_eoi = user_inputs_before_excess_of_information[
    user_inputs_before_excess_of_information['Errors_GOLD'].isin(accepted_user_errors) | user_inputs_before_excess_of_information['Errors_GOLD'].isnull()
]
# Extract only the text of the queries
final_user_queries_eoi = filtered_user_inputs_eoi['Text'].sample(4).tolist()
final_user_queries_eoi

In [ ]:
# Calcolare la lunghezza dell'array matched_queries_updated
array_length = len(filtered_user_inputs_eoi)
print("La lunghezza dell'array è:", array_length)

In [ ]:
filtered_user_inputs_eoi['Text'].tolist()

## Lack of Information

In [ ]:
# Identify rows where the system response has a "Lack of information" error
lack_of_information_indices = system_responses[system_responses['Errors_GOLD'] == 'Lack of information'].index

# Get the corresponding user inputs just before the system responses with "Lack of information"
user_inputs_before_lack_of_information = data.loc[lack_of_information_indices - 1]

# Define the accepted user errors
accepted_user_errors = ['Ignoring question/feedback', 'Grammatical error', 'Lack of information', 'Ill-formed']

# Filter the user inputs to include only those with the specified Errors_GOLD
filtered_user_inputs_loi = user_inputs_before_lack_of_information[
    user_inputs_before_lack_of_information['Errors_GOLD'].isin(accepted_user_errors) | user_inputs_before_lack_of_information['Errors_GOLD'].isnull()
]
# Extract only the text of the queries
final_user_queries_loi = filtered_user_inputs_loi['Text'].sample(4).tolist()
final_user_queries_loi

In [ ]:
# Calcolare la lunghezza dell'array matched_queries_updated
array_length = len(filtered_user_inputs_loi)
print("La lunghezza dell'array è:", array_length)

In [ ]:
filtered_user_inputs_loi['Text'].tolist()

# Chiamata al modello

In [ ]:
# Seleziono LLama2 13B come modello
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

# Download del modello da Hugging Face Hub
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# Inizializzazione del modello con configurazione per GPU
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # Numero di core della CPU
    n_batch=512,  # Numero di batch, dipende dalla VRAM della GPU
    n_gpu_layers=32  # Numero di layer gestiti dalla GPU
)
lcpp_llm.params.n_gpu_layers

In [ ]:
# Generazione della risposta
def call_model(prompt):
  response = lcpp_llm(prompt=prompt, max_tokens=4096)
  #print(response["choices"][0]["text"])
  return response["choices"][0]["text"]

# Salvataggio Dati

In [ ]:
def setup_xlsx(file_path):
    # Controlla se il file esiste
    if not os.path.exists(file_path):
        # Crea un nuovo workbook e aggiunge un foglio
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = "Data"
        ws.append(['Input', 'Translation', 'Output', 'Type', 'Error'])  # Aggiungi intestazioni
        wb.save(file_path)
    else:
        # Apri il workbook esistente
        wb = openpyxl.load_workbook(file_path)
    return wb

def append_to_xlsx(wb, file_path, data):
    # Carica il foglio di lavoro
    ws = wb.active
    # Aggiungi i nuovi dati
    ws.append(data)
    # Salva il workbook
    wb.save(file_path)

# Percorso del file Excel
filename = '/content/drive/My Drive/Llama2_Translation_LNC/Llama2InteractionCorpus_01_05_IR_EOI_LOI.xlsx'

# Setup iniziale del file Excel
wb = setup_xlsx(filename)

def update_excel(input_text, translation, output, prompt_type, error_type):
    # Dati da aggiungere
    data = [input_text, translation, output, prompt_type, error_type]
    # Aggiorna il file Excel
    append_to_xlsx(wb, filename, data)

# Prompt Analysis

In [ ]:
def extract_between_brackets(input_string):
    # Usa un'espressione regolare per trovare il testo tra < e >
    match = re.search(r'<([^>]*)>', input_string)
    # Se c'è una corrispondenza, restituisci il gruppo catturato, altrimenti restituisci None o una stringa vuota
    if match:
        return match.group(1)
    else:
        return "Output not found!"

In [ ]:
llama2PromptTemplate = lambda userPrompt,systemPrompt: f"""
<s>[INST] <<SYS>>
{systemPrompt}
<</SYS>>

{userPrompt} [/INST]
"""

In [ ]:
systemPrompt="""
You are an assistant expert on finite state automata. Your task is to translate \
user inputs from natural language to a controlled natural language format. \
This translation should adhere to specific vocabulary and syntax guidelines \
suitable for an AIML chatbot. Your response must be precise, using technical \
terminology related to finite state automata where appropriate. Ensure the \
translated text contains clear, unambiguous language that aligns with the \
rules governing the AIML chatbot's response system. Focus on maintaining the \
integrity of the technical content while simplifying the explanation for \
accurate and effective chatbot communication.
"""

In [ ]:
userPromptZeroShot = lambda text: f"""
  Translate the text delimited by triple backticks, without any preambles or \
  additional explanation, into controlled natural language sentence suitable \
  for an AIML system.
  Format the response by directly placing the translation within angle \
  brackets < >.
  For example:
  Output: <Your text here>.
  Now, use the same format for the translation:
  ``` {text} ```
  """

In [ ]:
userPromptChainOfThoughts = lambda text: f"""
Translate '{text}' into controlled natural language for an AIML chatbot, focusing on:
1. Identifying key components related to finite state automata.
2. Using precise, technical terminology where appropriate.
3. Ensuring clarity and adherence to AIML chatbot rules.
Ensure that only the response itself is literally placed between '<' and '>'
"""

In [ ]:
userPromptChainOfThoughts = lambda text: f"""
Translate the user's input considering the following steps:
1. Analyze the user input for key components:
   Input: '{text}'
2. Identify the critical elements and any technical terminology related to finite state automata:
   - Examine keywords and phrases that signify specific automata concepts.
3. Construct a response in controlled natural language:
   - Determine how to express these components clearly and unambiguously.
   - Use controlled language structures that fit the AIML chatbot's rules.
4. Synthesize the translation to ensure clarity, precision, and adherence to technical terms.
5. Finalize the translation ensuring technical accuracy and alignment with AIML standards:
   - Format the response to be enclosed within angular brackets.
6. Explicit instruction for output:
   - Example format for final output: 'Please format the final response like this: <RESPONSE>'
   - Ensure that only the response itself is literally placed between '<' and '>'.
Example:
Input: "Tell me a little bit about the automaton "
Output: "<Describe me briefly the automaton>"
"""

In [ ]:
userPromptZeroChainOfThoughts = lambda text: f"""
  Translate the text delimited by triple backticks, without any preambles or \
  additional explanation, into controlled natural language sentence suitable \
  for an AIML system.
  Format the response by directly placing the translation within angle \
  brackets < >.
  For example:
  Output: <Your text here>.
  Now the same format for the translation:
  ``` {text} ```
  Take a deep breath and work on this problem step-by-step.
  """

## Topic Change

### Zero Shot

In [ ]:
text = "Tell me a little bit about the automaton"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Shot", "Topic Change")

In [ ]:
# Zero-shot prompt focused on translating into controlled natural language
for query in final_user_queries_tc:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Shot", "Topic Change")

### One Shot

In [ ]:
userPromptOneShotTC = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the example:

Example:
Input: "Tell me a little bit about the automaton "
Output: "<Describe me briefly the automaton>"

Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.

Input: {text}
Output:
"""


In [ ]:
text = "what are the states and the connections between states"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptOneShotTC(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "One Shot", "Topic Change")

In [ ]:
# One-shot prompt focused on translating into controlled natural language
for query in final_user_queries_tc:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptOneShotTC(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "One Shot", "Topic Change")

### Few Shots

Example 1:
Input: "Tell me a little bit about the automaton"
Output: "Describe me briefly the automaton>"

Example 2:
Input: "According to your understanding of Automaton 2..."
Output: "Tell me more about the automaton 2>"

Example 3:
Input: "What is the final state of an automaton"
Output: "Identify the final state of the automaton>"

Example 4:
Input: "How are transitions defined"
Output: "What are the transitions>"

Example 5:
Input: "How is an automaton defined"
Output: "What is an automaton>"

Example 6:
Input: "what are the symbols accepted by the automaton"
Output: "Specify the alphabet used by the automaton.>"

Example 7:
Input: "nodes"
Output: "Tell me more about states>"

Example 8:
Input: "What are the transitions"
Output: "What are transitions>"

Example 9:
Input: "Give me a list of the arcs"
Output: "What are arcs>"

Example 10:
Input: "what is the initial stage of the automaton"
Output: "What is the initial state>"

In [ ]:
userPromptFewShotsTC = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the examples:
Example 1:
Input: "According to your understanding of Automaton 2..."
Output: "<Tell me more about the automaton 2>"
Example 2:
Input: "What is the final state of an automaton"
Output: "<Identify the final state of the automaton>"
Example 3:
Input: "How are transitions defined"
Output: "<What are the transitions>"
Example 4:
Input: "How is an automaton defined"
Output: "<What is an automaton>"
Example 5:
Input: "what are the symbols accepted by the automaton"
Output: "<Specify the alphabet used by the automaton.>"
Example 6:
Input: "nodes"
Output: "<Tell me more about states>"
Example 7:
Input: "Give me a list of the arcs"
Output: "<What are arcs>"
Example 8:
Input: "what is the initial stage of the automaton"
Output: "<What is the initial state>"
Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.
Input: {text}
Output:
"""

In [ ]:
text = "According to your understanding of Automaton 2 what is its optimal spatial representation"
print("\nInput: " + text)
model_output = call_model(llama2PromptTemplate(userPromptFewShotsTC(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Few Shots", "Topic Change")

In [ ]:
# Few-shots prompt focused on translating into controlled natural language
for query in final_user_queries_tc:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptFewShotsTC(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Few Shots", "Topic Change")

### Chain of Thoughts

In [ ]:
text = "what are the states and the connections between states"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
print("Completed Output: " + model_output)
#update_excel(text, translation, model_output, "Chain of Thoughts", "Topic Change")

In [ ]:
# Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_tc:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Chain of Thoughts", "Topic Change")

###Zero Chain of Thoughts

In [ ]:
text = "what are the states and the connections between states"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Topic Change")

In [ ]:
# Zero Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_tc:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Topic Change")

## Straight Wrong Response

### Zero Shot

In [ ]:
text = "Is there an arc between q0 and q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Shot", "Straight Wrong Response")

In [ ]:
# Zero-shot prompt focused on translating into controlled natural language
for query in final_user_queries_swr:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Shot", "Straight Wrong Response")

### One Shot

In [ ]:
userPromptOneShotSWR = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the example:

Example:
Input: "is q4 linked to q0"
Output: "<Is there a transition between q4 and q0>"

Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.

Input: {text}
Output:
"""

In [ ]:
text = "Is there an arc between q0 and q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptOneShotSWR(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "One Shot", "Straight Wrong Response")

In [ ]:
# One-shot prompt focused on translating into controlled natural language
for query in final_user_queries_swr:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptOneShotSWR(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "One Shot", "Straight Wrong Response")

### Few Shots

In [ ]:
userPromptFewShotsSWR = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the examples:
Example 1:
Input: "is q4 linked to q0"
Output: "<Is there a transition between q4 and q0>"
Example 2:
Input: "transitions"
Output: "<What are transitions>"
Example 3:
Input: "How are transitions defined"
Output: "<What are the transitions>"
Example 4:
Input: "arc from q2 to q1"
Output: "<Is there a transition between q2 and q1>"
Example 5:
Input: "q0"
Output: "<What is q0>"
Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.
Input: {text}
Output:
"""

In [ ]:
text = "Is there an arc between q0 and q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptFewShotsSWR(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Few Shots", "Straight Wrong Response")

In [ ]:
# Few-shots prompt focused on translating into controlled natural language
for query in final_user_queries_swr:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptFewShotsSWR(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Few Shots", "Straight Wrong Response")

### Chain of Thoughts

In [ ]:
text = "Is there an arc between q0 and q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Chain of Thoughts", "Straight Wrong Response")

In [ ]:
# Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_swr:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Chain of Thoughts", "Straight Wrong Response")

###Zero Chain of Thoughts

In [ ]:
text = "Is there an arc between q0 and q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Straight Wrong Response")

In [ ]:
# Zero Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_swr:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Straight Wrong Response")

## Indirect Response

### Zero Shot

In [ ]:
text = "is there an arc from q0 to q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Shot", "Indirect Response")

In [ ]:
# Zero-shot prompt focused on translating into controlled natural language
for query in final_user_queries_ir:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Shot", "Indirect Response")

### One Shot

In [ ]:
userPromptOneShotIR = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the example:

Example:
Input: "is there an arc from q0 to q1"
Output: "<Is there a transition between q4 and q0>"

Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.

Input: {text}
Output:
"""


In [ ]:
text = "is there an arc from q0 to q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptOneShotIR(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "One Shot", "Indirect Response")

In [ ]:
# One-shot prompt focused on translating into controlled natural language
for query in final_user_queries_ir:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptOneShotIR(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "One Shot", "Indirect Response")

### Few Shots

In [ ]:
userPromptFewShotsIR = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the examples:
Example 1:
Input: "is there an arc from q0 to q1"
Output: "<Is there a transition between q4 and q0>"
Example 2:
Input: "is q4 a final state"
Output: "<What is q4>"
Example 3:
Input: "The automaton accepts a language allowing words made of an odd number of 0s and 1s"
Output: "<The automaton's alphabet consists of words with an odd number of 0s and 1s>"
Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.
Input: {text}
Output:
"""

In [ ]:
text = "is there an arc from q0 to q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptFewShotsIR(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Few Shots", "Indirect Response")

In [ ]:
# Few-shots prompt focused on translating into controlled natural language
for query in final_user_queries_ir:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptFewShotsIR(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Few Shots", "Indirect Response")

### Chain of Thoughts

In [ ]:
text = "is there an arc from q0 to q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Chain of Thoughts", "Indirect Response")

In [ ]:
# Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_ir:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Chain of Thoughts", "Indirect Response")

###Zero Chain of Thoughts

In [ ]:
text = "is there an arc from q0 to q1"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Indirect Response")

In [ ]:
# Zero Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_ir:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Indirect Response")

## Excess of Information

### Zero Shot

In [ ]:
text = "how is marked the arc between q0 and q2"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Shot", "Excess of Information")

In [ ]:
# Zero-shot prompt focused on translating into controlled natural language
for query in final_user_queries_eoi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Shot", "Excess of Information")

### One Shot

In [ ]:
userPromptOneShotEOI = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the example:

Example:
Input: "Please describe the transitions"
Output: "<What are transitions>"

Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.

Input: {text}
Output:
"""


In [ ]:
text = "how is marked the arc between q0 and q2"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptOneShotEOI(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "One Shot", "Excess of Information")

In [ ]:
# One-shot prompt focused on translating into controlled natural language
for query in final_user_queries_eoi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptOneShotEOI(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "One Shot", "Excess of Information")

### Few Shots

In [ ]:
userPromptFewShotsEOI = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the examples:
Example 1:
Input: "Please describe the transitions"
Output: "<What are transitions>"
Example 2:
Input: "how is marked the arc between q0 and q2"
Output: "<Identify the label on the arc between q0 and q2>"
Example 3:
Input: "which is final state"
Output: "<Tell me more about the final state>"
Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.
Input: {text}
Output:
"""

In [ ]:
text = "how is marked the arc between q0 and q2"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptFewShotsEOI(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Few Shots", "Excess of Information")

In [ ]:
# Few-shots prompt focused on translating into controlled natural language
for query in final_user_queries_eoi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptFewShotsEOI(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Few Shots", "Excess of Information")

### Chain of Thoughts

In [ ]:
text = "how is marked the arc between q0 and q2"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Chain of Thoughts", "Excess of Information")

In [ ]:
# Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_eoi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Chain of Thoughts", "Excess of Information")

###Zero Chain of Thoughts

In [ ]:
text = "how is marked the arc between q0 and q2"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Excess of Information")

In [ ]:
# Zero Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_eoi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Excess of Information")

## Lake of Information

### Zero Shot

In [ ]:
text = "What transitions enter and exit q0"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Shot", "Lake of Information")

In [ ]:
# Zero-shot prompt focused on translating into controlled natural language
for query in final_user_queries_loi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroShot(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Shot", "Lake of Information")

### One Shot

In [ ]:
userPromptOneShotLOI = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the example:

Example:
Input: "Briefly describe the automaton How many states are there"
Output: "<Describe me briefly the automaton and tell me how many states are there>"

Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.

Input: {text}
Output:
"""


In [ ]:
text = "What transitions enter and exit q0"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptOneShotLOI(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "One Shot", "Lake of Information")

In [ ]:
# One-shot prompt focused on translating into controlled natural language
for query in final_user_queries_loi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptOneShotLOI(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "One Shot", "Lake of Information")

### Few Shots

In [ ]:
userPromptFewShotsLOI = lambda text: f"""
Please translate the following inputs into controlled natural language.
Use the format provided in the examples:
Example 1:
Input: "Briefly describe the automaton How many states are there"
Output: "<Describe me briefly the automaton and tell me how many states are there>"
Example 2:
Input: "There is a transition between q2 and q0"
Output: "<Is there a transition between q2 and q0>"
Example 3:
Input: "What transitions enter and exit q1"
Output: "<List transitions entering and exiting q1>"
Now, translate the new input using the same controlled natural language format.
Format the response by directly placing the translation within angle brackets < >.
Input: {text}
Output:
"""

In [ ]:
text = "What transitions enter and exit q0"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptFewShotsLOI(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Few Shots", "Lake of Information")

In [ ]:
# Few-shots prompt focused on translating into controlled natural language
for query in final_user_queries_loi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptFewShotsLOI(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Few Shots", "Lake of Information")

### Chain of Thoughts

In [ ]:
text = "What transitions enter and exit q0"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Chain of Thoughts", "Lake of Information")

In [ ]:
# Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_loi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Chain of Thoughts", "Lake of Information")

###Zero Chain of Thoughts

In [ ]:
text = "What transitions enter and exit q0"
print("Input: " + text)
model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
translation = extract_between_brackets(model_output)
print("Output: " + translation)
update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Lake of Information")

In [ ]:
# Zero Chain of Thoughts prompt focused on translating into controlled natural language
for query in final_user_queries_loi:
  text = query
  print("\nInput: " + text)
  model_output = call_model(llama2PromptTemplate(userPromptZeroChainOfThoughts(text), systemPrompt))
  translation = extract_between_brackets(model_output)
  print("Output: " + translation)
  update_excel(text, translation, model_output, "Zero Chain of Thoughts", "Lake of Information")